<a href="https://colab.research.google.com/github/LarsBentsen/CourseDSAIStatisticalLearning/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting House Prices

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Load the California Housing Dataset

In [ ]:
df = pd.read_csv('https://github.com/LarsBentsen/CourseDSAIStatisticalLearning/blob/main/data/California_Housing.txt?raw=true')
print("number of rows: ", df.shape[0])
df.head()

In [ ]:
# rename columns to be consistent with the book
df.rename(columns={
    'population': 'Population',
    'housingMedianAge': 'HouseAge',
    'longitude': 'Longitude',
    'latitude': 'Latitude',
    'medianIncome': 'MedInc',
    'medianHouseValue': 'MedHouseVal'}, inplace=True)
df['MedHouseVal'] /= 100000

# calculate average values from total values
df['AveBedrms'] = df['totalBedrooms'] / df['households']
df['AveRooms'] = df['totalRooms'] / df['households']
df['AveOccup'] = df['Population'] / df['households']

# The response variable Y is the median house value in each
# neighborhood measured in units of $100,000.
target = 'MedHouseVal'

features = ['Population', 'AveBedrms', 'AveRooms', 'HouseAge',
            'Latitude', 'AveOccup', 'Longitude', 'MedInc']

In [ ]:
# check missing values
df.info()

## Plot the correlation matrix to get a sense of the linear correlation between features

In [ ]:
corr = df.corr()
# Plot the correlation matrix
fig, ax = plt.subplots(figsize=(5, 4))
cax = ax.matshow(corr, cmap='coolwarm')

# Add color bar
plt.colorbar(cax)

# Set ticks and labels
ax.set_xticks(np.arange(len(corr.columns)))
ax.set_yticks(np.arange(len(corr.columns)))

ax.set_xticklabels(corr.columns, rotation=90)
ax.set_yticklabels(corr.columns)

# Customize the x-tick label for "MedHouseVal"
for label in ax.get_xticklabels():
    if label.get_text() == "MedHouseVal":
        label.set_color('red')

# Customize the y-tick label for "MedHouseVal"
for label in ax.get_yticklabels():
    if label.get_text() == "MedHouseVal":
        label.set_color('red')

# Show the plot
plt.show()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# Scatter plots of the features against each other and histograms of the features
plt.figure(figsize=(5,5))
sns.pairplot(df)
plt.show()

## Data preperation

Before model training, data needs to be prepared to be suitable for further model training and analysis

### Longitude
Example with one feature. Check distribution and correlation with target feature. Will decide if this column can be dropped. 

In [ ]:
df['Longitude'].describe()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(x=df['Longitude'], bins=10, palette='viridis', kde=True)
plt.title("The distribution of Longitude Feature")
plt.ylabel("")
plt.show()

In [ ]:
# Check the correlation between the feature and response (median house value)
# Longitude does not contain any useful information for median house value
plt.figure(figsize=(6,5))
sns.scatterplot(x=df['Longitude'], y=df['MedHouseVal'])

### Average of total rooms

In [ ]:
df['AveBedrms'].describe()

In [ ]:
plt.figure(figsize=(7,6))
sns.histplot(x=df['AveBedrms'], bins=10, kde=True)
plt.title("The distribution of the average total rooms")
plt.ylabel("")
plt.show()

Strong right-skewed feature.
Check possible outliers

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x=df['AveBedrms'])
plt.title("The representation of the possible outliers")
plt.show()

Many possible outliers. Choose a threshold to filter them out.
Test with the 95th percentile

In [ ]:
perc_99 = np.percentile(df['AveBedrms'], 99)
print(f'99th Percentile Value: {perc_99}')
outliers = (df['AveBedrms']>perc_99).sum()
print(f'Number of Potential Outliers: {outliers}')
print(f"The max value is {np.max(df['AveBedrms'])}")

In [ ]:
# drop values above the 95th percentile
df.drop(df[df['AveBedrms']>perc_99].index, axis=0, inplace=True)

In [ ]:
# check distribution
plt.figure(figsize=(7,6))
sns.histplot(x=df['AveBedrms'], bins=10, kde=True)
plt.title("The distribution of the average total rooms after remove outliers")
plt.ylabel("")
plt.show()

In [ ]:
# check distribution
plt.figure(figsize=(7,6))
sns.histplot(x=df['AveOccup'], bins=10, kde=True)
plt.title("The distribution of the average occupants")
plt.show()

In [ ]:
perc_99 = np.percentile(df['AveOccup'], 99)
print(f'95th Percentile Value: {perc_99}')
outliers = (df['AveOccup']>perc_99).sum()
print(f'Number of Potential Outliers: {outliers}')
print(f"The max value is {np.max(df['AveOccup'])}")

In [ ]:
# drop values above the 95th percentile
df.drop(df[df['AveOccup']>perc_99].index, axis=0, inplace=True)

In [ ]:
# check distribution
plt.figure(figsize=(7,6))
sns.histplot(x=df['AveOccup'], bins=10, kde=True)
plt.title("The distribution of the average occupants")
plt.show()

Feature is now more normally distributed. Ideally you want to do this for all the columns, but will be skipped here for brevity... 

# Model training

In [ ]:
# Split Test/Train
# Set the seed (so that we all get the same results)
np.random.seed(666)
test_indxs = np.random.choice(np.arange(df.shape[0]), size=df.shape[0] // 5, replace=False)
df_test = df.iloc[test_indxs]
df = df.drop(df.index[test_indxs])

In [ ]:
# Start with the simplest model - linear regression

# divide target value from features
X_train = df[features]
y_train = df[target]

X_test = df_test[features]
y_test = df_test[target]

lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
# Model evaluation
print(f"Train Accuracy:{lr.score(X_train,y_train)}")        # i.e. R2 score
print(f"Test Accuracy:{lr.score(X_test,y_test)}")

y_pred = lr.predict(X_test)
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}")
print(f"R2 Score: {r2_score(y_test, y_pred)}")

In [ ]:
mean_squared_error(y_train,  lr.predict(X_train))

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, y_pred, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(y_pred), max(y_test))
p2 = min(min(y_pred), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:

lr = LinearRegression()
lr.fit(X_train,y_train)

fig, axs = plt.subplots(2, int(np.ceil(len(features)/2)), figsize=(20, 10))
axs = axs.flatten()
for i, f_i in enumerate(features):
    axs[i].set_title(f_i)
    axs[i].scatter(X_train[f_i], y_train.values - lr.predict(X_train), alpha=0.02)

In [ ]:
# Define the models and their parameter grids for tuning
param_grid_lasso = {'alpha': np.logspace(-4, 4, 50)}
param_grid_ridge = {'alpha': np.logspace(-4, 4, 50)}

# Define 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV for Lasso and Ridge
lasso_cv = GridSearchCV(Lasso(), param_grid_lasso, cv=kf, scoring='neg_mean_squared_error')
ridge_cv = GridSearchCV(Ridge(), param_grid_ridge, cv=kf, scoring='neg_mean_squared_error')

# Fit the models
lasso_cv.fit(X_train, y_train)
ridge_cv.fit(X_train, y_train)

# Print the best parameters and corresponding scores
print(f"Best Lasso alpha: {lasso_cv.best_params_['alpha']}")
print(f"Best Lasso MSE: {-lasso_cv.best_score_}")

print(f"Best Ridge alpha: {ridge_cv.best_params_['alpha']}")
print(f"Best Ridge MSE: {-ridge_cv.best_score_}")

# Optionally, fit the models on the entire dataset with the best found parameters
best_lasso = Lasso(alpha=lasso_cv.best_params_['alpha']).fit(X_train, y_train)
best_ridge = Ridge(alpha=ridge_cv.best_params_['alpha']).fit(X_train, y_train)

print(f"Best Lasso model coefficients: {best_lasso.coef_}")
print(f"Best Ridge model coefficients: {best_ridge.coef_}")

In [ ]:

# Make predictions with all models
y_pred_lr = LinearRegression().fit(X_train, y_train).predict(X_test)
y_pred_lasso = best_lasso.predict(X_test)
y_pred_ridge = best_ridge.predict(X_test)


# Plot predictions
plt.figure(figsize=(10,10))
plt.scatter(y_test, y_pred_lr, c='blue', label='Linear Regression', alpha=0.3)
plt.scatter(y_test, y_pred_lasso, c='red', label='Lasso', alpha=0.3)
plt.scatter(y_test, y_pred_ridge, c='green', label='Ridge', alpha=0.3)
plt.yscale('log')
plt.xscale('log')
p1 = max(max(y_pred_lr), max(y_test))
p2 = min(min(y_pred_lr), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.legend()
plt.show()

# Model evaluation
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": best_lasso,
    "Ridge": best_ridge
}


results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    train_mse = mean_squared_error(y_train, model.predict(X_train))
    test_mse = mean_squared_error(y_test, model.predict(X_test))
    test_mae = mean_absolute_error(y_test, model.predict(X_test))
    test_r2 = r2_score(y_test, model.predict(X_test))
    results.append({'Model': name, 'Train MSE': train_mse, 'Test MSE': test_mse, 
                    'Test MAE': test_mae, 'Test R2': test_r2})

results_df = pd.DataFrame(results)
print(results_df)

# Your turn: 

1. Can we improve the preprocessing of the data? Try to investigate the other features to see if you can find any other columns with significant outliers. 
2. Can you improve performance by trying to avoid heteroscedastic errors? 
    - Are there any trends in the residuals or residual variance? 
3. Is a more advanced model maybe required? Maybe try a boosting model, SVR or MLP? 